In [16]:
import numpy as np
import time 
import random

import gym, torch
import argparse
import pickle
from core.operator_runner import OperatorRunner
from parameters import Parameters

import logging
import sys
import os

class Config:
    def __init__(self, config=None):
        if config is None:
            config = {}
        self.env = config.get('env', "Swimmer-v2")
        self.seed = config.get('seed', 7)
        self.pr = config.get('pr', 64)
        self.pop_size = config.get('pop_size', 5)
        self.disable_cuda = config.get('disable_cuda', False)
        self.render = config.get('render', False)
        self.sync_period = config.get('sync_period', None)
        self.novelty = config.get('novelty', False)
        self.proximal_mut = config.get('proximal_mut', False)
        self.distil = config.get('distil', False)
        self.distil_type = config.get('distil_type', 'fitness')
        self.EA = config.get('EA', True)
        self.RL = config.get('RL', True)
        self.detach_z = config.get('detach_z', False)
        self.random_choose = config.get('random_choose', False)
        self.per = config.get('per', False)
        self.use_all = config.get('use_all', False)
        self.intention = config.get('intention', False)
        self.mut_mag = config.get('mut_mag', 0.05)
        self.tau = config.get('tau', 0.005)
        self.prob_reset_and_sup = config.get('prob_reset_and_sup', 0.05)
        self.frac = config.get('frac', 1)
        self.TD3_noise = config.get('TD3_noise', 0.2)
        self.mut_noise = config.get('mut_noise', False)
        self.verbose_mut = config.get('verbose_mut', False)
        self.verbose_crossover = config.get('verbose_crossover', False)
        self.opstat = config.get('opstat', False)
        self.opstat_freq = config.get('opstat_freq', 1)
        self.save_periodic = config.get('save_periodic', False)
        self.next_save = config.get('next_save', 200)
        self.K = config.get('K', 1)
        self.n_nets = config.get('n_nets', 4)
        self.bellman_mode = config.get('bellman_mode', "TV")
        self.mutate_mode = config.get('mutate_mode', "distill_mutate")
        self.OFF_TYPE = config.get('OFF_TYPE', 1)
        self.num_evals = config.get('num_evals', 1)
        self.version = config.get('version', 1)
        self.time_steps = config.get('time_steps', 200)
        self.repair_name = config.get('repair_name', "")
        self.device = config.get('device', "cuda:0")
        self.test_operators = config.get('test_operators', False)
        self.EA_actor_alpha = config.get('EA_actor_alpha', 1.0)
        self.state_alpha = config.get('state_alpha', 0.0)
        self.actor_alpha = config.get('actor_alpha', 1.0)
        self.theta = config.get('theta', 0.2)
        self.gamma = config.get('gamma', 0.99)
        self.state_dim = None
        self.action_dim = None
        self.max_action = None

args = {
    'env': "Ant-v2",
    'seed': 7,
    'pr': 64,
    'pop_size': 5,
    'disable_cuda': False,
    'render': False,
    'sync_period': None,
    'novelty': False,
    'proximal_mut': False,
    'distil': False,
    'distil_type': 'fitness',
    'EA': True,
    'RL': True,
    'detach_z': False,
    'random_choose': False,
    'per': False,
    'use_all': False,
    'intention': False,
    'mut_mag': 0.05,
    'tau': 0.005,
    'prob_reset_and_sup': 0.05,
    'frac': 1,
    'TD3_noise': 0.2,
    'mut_noise': False,
    'verbose_mut': False,
    'verbose_crossover': False,
    'opstat': False,
    'opstat_freq': 1,
    'save_periodic': False,
    'next_save': 200,
    'K': 1,
    'n_nets': 4,
    'bellman_mode': "TV",
    'mutate_mode': "distill_mutate",
    'OFF_TYPE': 1,
    'num_evals': 1,
    'version': 1,
    'time_steps': 200,
    'repair_name': "",
    'device': "cuda:0",
    'test_operators': False,
    'EA_actor_alpha': 1.0,
    'state_alpha': 0.0,
    'actor_alpha': 1.0,
    'theta': 0.2,
    'gamma': 0.99,
}

parameters = Config(args)   

env = gym.make('Ant-v2')
parameters.action_dim = env.action_space.shape[0]
parameters.state_dim = env.observation_space.shape[0]
parameters.max_action = env.action_space.high[0]
operator_runner = OperatorRunner(parameters, env)
operator_runner.run()

========== Crossover between 1400 and 1600 ==============


FileNotFoundError: [Errno 2] No such file or directory: 'exp/cheetah_sm0.1_distil_save_20/models/evo_net_critic_1600.pkl'